# Setup

### Find paths

In [31]:
repo_path = !git rev-parse --show-toplevel
assert (len(repo_path) == 1)

exec_path = repo_path[0] + "/benchmark_process_packet.exe"
exec_path

'/home/bm611/ebsl/benchmark_process_packet.exe'

### Run benchmark function

In [32]:
import numpy as np
import json

tasks = 50_000

def run(scheduler,dist_policy, domains=-1,spawners=-1):
    cmd = (exec_path + " -scheduler " + scheduler + " -num-of-domains " + str(domains) + 
        " -num-of-spawners " + str(spawners) + " -items-total " + str(tasks) + 
        " -dist-policy " + dist_policy)
    print(cmd)
    output = !$cmd
    
    formatted = ""
    try:
      formatted = json.loads(''.join(output))
    except:
      print("An exception occurred, data:")
      print(output)
      raise
    
    return formatted

In [33]:
def latency_undo_exp(x):
    return 1.2 ** x 

In [34]:
def find_percentile(array, percentile):
    assert (0 < percentile and percentile < 1)
    
    total = np.sum(array)
    
    percentile_boundary = percentile * total 
    current_total = 0
    for i in range(0,len(array)):
        current_total += array[i]
        if current_total > percentile_boundary:
            return i
    
    return "reached the end before finding percentile boundary"


def transform(parsed):
    time = [] 
    latency_half = []
    latency_threenine = []
    waited_for_space_on_enque = [] 
    latency = []
    latency_q10 = []
    latency_q25 = []
    latency_q50 = []
    latency_q99 = []
    latency_q999 = [] 
    latency_q9999 = []
    latency_q99999 = []

    for row in parsed['data']:
        if row['iteration'] == 1:
            continue

        time.append(row['time'])
        latency_half.append(latency_undo_exp(row['latency_median']))
        latency_threenine.append(latency_undo_exp(row['latency_three_nine']))
        waited_for_space_on_enque.append(row['waited-for-space-on-enque'])
        latency.append(row['latency'])
        
        latency_q50.append(latency_undo_exp(find_percentile(row['latency'], 0.5)))
        latency_q99.append(latency_undo_exp(find_percentile(row['latency'], 0.99)))
        latency_q999.append(latency_undo_exp(find_percentile(row['latency'], 0.999)))
        latency_q9999.append(latency_undo_exp(find_percentile(row['latency'], 0.9999)))
        latency_q99999.append(latency_undo_exp(find_percentile(row['latency'], 0.99999)))
                

    return {'time':time, 
            'waited_for_space_on_enque':waited_for_space_on_enque,
            'latency':latency,
            'latency_q50':latency_q50,
            'latency_q99':latency_q99,
            'latency_q999':latency_q999,
            'latency_q9999':latency_q9999,
            'latency_q99999':latency_q99999,
            'latency_half':latency_half,
            'median_latency_threenine':latency_threenine,
           }

In [35]:
def to_median_and_errs(array):
    median = np.median(array)
    lower_err = median - np.quantile(array, 0.25)
    upper_err = np.quantile(array, 0.75) - median
    return {'median': median, 'lower_err': lower_err, 'upper_err': upper_err}
    
def add_metrics(data):
    data['time-processed'] = to_median_and_errs(data['time'])
    data['latency_half-processed'] = to_median_and_errs(data['latency_half'])
    data['latency_threenine-processed'] = to_median_and_errs(data['median_latency_threenine'])
    data['waited_for_space_on_enque-processed'] = to_median_and_errs(data['waited_for_space_on_enque'])
    
    data['throughput-processed'] = to_median_and_errs((tasks * 1_000.)/np.array(data['time']))
    
    data['latency_q50-processed'] = to_median_and_errs(data['latency_q50'])
    data['latency_q99-processed'] = to_median_and_errs(data['latency_q99'])
    data['latency_q999-processed'] = to_median_and_errs(data['latency_q999'])
    data['latency_q9999-processed'] = to_median_and_errs(data['latency_q9999'])
    data['latency_q99999-processed'] = to_median_and_errs(data['latency_q99999'])
    
    

### Find percentiles function

### Plotting

In [36]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

# Experiment 1: simple workload, skewed

## Data collection

In [41]:
param_space = range(10,120,20)

policies = ["steal", "steal_slowed", "steal_and_mpmc_overflow", 
            "steal_localized", "steal_localized_small",
            "steal_and_multi_mpmc_overflow", "steal_and_advanced_request"]
fifos = []
#fifos_res = [] 
lifos = []

for policy in policies:

    fifo = []
    for i in param_space:
        fifo.append(run("FIFO", policy, domains=i, spawners=1))
    fifos.append(fifo)
        
    lifo = []
    for i in param_space:
        lifo.append(run("LIFO", policy, domains=i, spawners=1))
    lifos.append(lifo)

    
x = list(param_space)

/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 5 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 15 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 25 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 35 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 45 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 55 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 65 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_

/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 95 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_mpmc_overflow
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 105 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_mpmc_overflow
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO -num-of-domains 115 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_mpmc_overflow
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 5 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_mpmc_overflow
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 15 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_mpmc_overflow
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 25 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_mpmc_overflow
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -

/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 35 -num-of-spawners 1 -items-total 50000 -dist-policy steal_localized_small
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 45 -num-of-spawners 1 -items-total 50000 -dist-policy steal_localized_small
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 55 -num-of-spawners 1 -items-total 50000 -dist-policy steal_localized_small
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 65 -num-of-spawners 1 -items-total 50000 -dist-policy steal_localized_small
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 75 -num-of-spawners 1 -items-total 50000 -dist-policy steal_localized_small
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 85 -num-of-spawners 1 -items-total 50000 -dist-policy steal_localized_small
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domain

/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 75 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_advanced_request
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 85 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_advanced_request
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 95 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_advanced_request
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 105 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_advanced_request
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler LIFO -num-of-domains 115 -num-of-spawners 1 -items-total 50000 -dist-policy steal_and_advanced_request


In [38]:
res_policies = ["steal", "steal_and_advanced_request"]

In [39]:
fifos_res = []
for policy in res_policies:
    fifo_res = []
    for i in param_space:
        fifo_res.append(run("FIFO_resize", policy, domains=i, spawners=1))
    fifos_res.append(fifo_res)
        

/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO_resize -num-of-domains 5 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO_resize -num-of-domains 15 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO_resize -num-of-domains 25 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO_resize -num-of-domains 35 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO_resize -num-of-domains 45 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO_resize -num-of-domains 55 -num-of-spawners 1 -items-total 50000 -dist-policy steal
/home/bm611/ebsl/benchmark_process_packet.exe -scheduler FIFO_resize -num-of-domains 65 -num-of-spawners 1 -items-total 50000 -dist

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 1269 (char 1268)

In [ ]:
run_name = "distribution6"

In [ ]:
import pickle


with open('packet_fifos_' + run_name + '.pickle', 'wb') as handle:
    pickle.dump(fifos, handle)

with open('packet_fifos_res' + run_name + '.pickle', 'wb') as handle:
    pickle.dump(fifos_res, handle)
    
with open('packet_lifos_' + run_name + '.pickle', 'wb') as handle:
    pickle.dump(lifos, handle)


In [ ]:
fifos_transformed = []

for fifo in fifos:
    fifo_transformed = []
    for row in fifo:
        fifo_transformed.append(transform(row))
    fifos_transformed.append(fifo_transformed)
    
fifos_res_transformed = [] 
for fifo_res in fifos_res:
    fifo_res_transformed = []
    for row in fifo_res:
        fifo_res_transformed.append(transform(row))
    fifos_res_transformed.append(fifo_res_transformed)
    
lifos_transformed = []
for lifo in lifos:
    lifo_transformed = []
    for row in lifo:
        lifo_transformed.append(transform(row))
    lifos_transformed.append(lifo_transformed)

In [ ]:
for fifo_transformed in fifos_transformed:
    for row in fifo_transformed:
        add_metrics(row)

for fifo_res_transformed in fifos_res_transformed:
    for row in fifo_res_transformed:
        add_metrics(row)
    
for lifo_transformed in lifos_transformed:
    for row in lifo_transformed:
        add_metrics(row)


## Analysis

### Exec time vs number of domains

In [ ]:
def to_series(data, label):
    return [v[label]["median"] for v in data]

In [ ]:
def plot(x, data, key, name):
    y = [v[key]["median"] for v in data]
    lower_errbar = [v[key]["lower_err"] for v in data]
    upper_errbar = [v[key]["upper_err"] for v in data]
    
    return ax.errorbar(x, y, [lower_errbar, upper_errbar], label=name)
    

In [42]:
size = 6

fig, axes = plt.subplots(size, 1, sharex=True)
fig.set_size_inches(12, 4 * size)


ax = axes[0]

#line_lifo, = ax.plot(x, to_series(fifo_transformed, "time-processed"), label='FIFO')
#line_fifo, = ax.plot(x, to_series(lifo_transformed, "time-processed"), label='LIFO')

handles = []
for i in range(0, len(policies)):
    handle = plot(x, fifos_transformed[i], "throughput-processed", 'FIFO ' + policies[i])
    handles.append(handle)
    
ax.legend(handles=handles)

ax.set_yscale("log")
#ax.set_ylim(ymin=2*10**5, ymax=1.5*10**7)
ax.set_title("Task throughput of packet benchmark over varying number of cores.") 
ax.set_ylabel('throughput (task/s)')

######################################
ax = axes[1]

#line_lifo, = ax.plot(x, to_series(fifo_transformed, "time-processed"), label='FIFO')
#line_fifo, = ax.plot(x, to_series(lifo_transformed, "time-processed"), label='LIFO')

handles = []

for i in range(0, len(policies)):
    handle = plot(x, lifos_transformed[i], "throughput-processed", 'LIFO ' + policies[i])
    handles.append(handle)
    
ax.legend(handles=handles)

ax.set_yscale("log")
#ax.set_ylim(ymin=2*10**5, ymax=1.5*10**7)
ax.set_title("Task throughput of packet benchmark over varying number of cores.") 
ax.set_ylabel('throughput (task/s)')
#################################

ax = axes[2]

#line_lifo, = ax.plot(x, to_series(fifo_transformed, "time-processed"), label='FIFO')
#line_fifo, = ax.plot(x, to_series(lifo_transformed, "time-processed"), label='LIFO')

handles = []

for i in range(0, len(res_policies)):
    handle = plot(x, fifos_res_transformed[i], "throughput-processed", 'FIFO_RES ' + res_policies[i])
    handles.append(handle)
    
ax.legend(handles=handles)

ax.set_yscale("log")
#ax.set_ylim(ymin=2*10**5, ymax=1.5*10**7)
ax.set_title("Task throughput of packet benchmark over varying number of cores.") 
ax.set_ylabel('throughput (task/s)')



'''
########################
## TIME ################
########################
ax = axes[1]
#line_lifo, = ax.plot(x, to_series(fifo_transformed, "time-processed"), label='FIFO')
#line_fifo, = ax.plot(x, to_series(lifo_transformed, "time-processed"), label='LIFO')

line_lifo = plot(x, fifo_transformed, "time-processed", 'FIFO')
line_fifo = plot(x, lifo_transformed, "time-processed", 'LIFO')

ax.set_ylim(ymin=1)
ax.set_title("Time to complete the benchmark.") 
ax.set_ylabel('execution time (ms)')


########################
## MEDIAN LATENCY ######
########################

ax = axes[2]


line_lifo = plot(x, fifo_transformed, "latency_q50-processed", 'FIFO')
line_fifo = plot(x, lifo_transformed, "latency_q50-processed", 'LIFO')


#ax.set_yscale("log")
ax.set_ylim(ymin=1)
ax.set_ylabel('latency (μs)')
ax.set_title("Median latency.") 

########################
## Q99 LATENCY ######
########################

ax = axes[3]


line_lifo = plot(x, fifo_transformed, "latency_q999-processed", 'FIFO')
line_fifo = plot(x, lifo_transformed, "latency_q999-processed", 'LIFO')

#ax.set_yscale("log")
ax.set_ylim(ymin=1)
ax.set_ylabel('latency (μs)')
ax.set_title("99.9% percentile latency.") 


########################
## Q99.9 LATENCY ######
########################

ax = axes[4]


line_lifo = plot(x, fifo_transformed, "latency_q9999-processed", 'FIFO')
line_fifo = plot(x, lifo_transformed, "latency_q9999-processed", 'LIFO')

#ax.set_yscale("log")
ax.set_ylim(ymin=1)
ax.set_ylabel('latency (μs)')
ax.set_title("99.99% percentile latency.") 

########################
## WAIT ################
########################

ax = axes[5]


line_lifo = plot(x, fifo_transformed, "waited_for_space_on_enque-processed", 'FIFO')
line_fifo = plot(x, lifo_transformed, "waited_for_space_on_enque-processed", 'LIFO')

ax.set_yscale("log")
ax.set_ylim(ymin=1)
ax.set_xlabel('domains count')
ax.set_ylabel('busy-wait loops (count)')
ax.set_title("Number of retries on task insertion due to lack of space") 

'''

ax.set_xlabel('domains count')

plt.savefig("packet-basic-" + run_name + ".png", dpi=200)